# **富邦新一代API - Python範例程式碼**

# 安裝

## 若已安裝過 SDK，可**忽略**此部分

### 安裝新一代API Python SDK
此範例使用Windows版本檔案 <br>
***註:*** SDK檔案名稱 (ex. fubon_neo-1.0.3-cp37-abi3-win_amd64.whl) 請根據實際下載.whl名稱修改 <br>
***註2:*** SDK檔案需與本jupyter notebook檔案至於同一資料夾內，或是修改以下指令之SDK檔案路徑

In [ ]:
# Pip install SDK
!pip install --force-reinstall --no-cache fubon_neo-2.0.1-cp37-abi3-win_amd64.whl

# SDK 版本檢視

In [8]:
import fubon_neo

In [9]:
fubon_neo.__version__

'2.0.1'

# 準備

In [10]:
 # 匯入 SDK Library
from fubon_neo.sdk import FubonSDK, Order
from fubon_neo.constant import TimeInForce, OrderType, PriceType, MarketType, BSAction

In [11]:
# 連結 API Server
sdk = FubonSDK()

## 登入

In [12]:
accounts = sdk.login("A121714564", "tzj1nez2wyyy", "C:\CAFubon\A121714564/A121714564.pfx", "tzj1nez2wyyy")   # 登入帳號 輸入:帳號、密碼、憑證路徑、憑證密碼
print(accounts)

Result {
  is_success: True,
  message: None,
  data: [Account {
    name: "李定洲",
    branch_no: "20505",
    account: "37089",
    account_type: "stock",
}]
}


# 行情元件

## **建立行情元件連線**

In [13]:
sdk.init_realtime() # 建立行情元件連線

## 行情 WebSocket 報價

### WebSocket callback functions 設定

In [14]:
import json
import traceback

subscribe_ids = []  # 訂閱頻道 id 列表

def handle_connect():  # 連線成功 callback
    print("行情連接成功")

def handle_disconnect(code, message):  # 連接斷線 callback
    print(f"行情連接斷線: {code}, {message}")

def handle_message(message): # 處理接收訊息 callback
    try:
        msg = json.loads(message)
        event = msg["event"]
        data = msg["data"]
    
        if event == "pong":
            return
        
        if event == "subscribed":
            id = data["id"]
            
            if id in subscribe_ids:
                print(f"Error: 訂閱 id {id} 已存在列表中")
            else:
                subscribe_ids.append(id)
    
        elif event == "unsubscribed":
            id = data["id"]
            
            try:
                subscribe_ids.remove(id)
            except:
                print(f"Error: 查無此筆訂閱 id 資料, id {id}")
    
        print(f'market data message: {message}')           
        
    except Exception as e:
        handle_error(f'Error parsing JSON: {e}', traceback.format_exc())

def handle_error(error,traceback_info=None):  # 處理程式錯誤訊息 callback
    print(f'market data error: {error}')
    if traceback_info:
        print(f'Traceback:\n{traceback_info}')

stock = sdk.marketdata.websocket_client.stock
stock.on("connect", handle_connect)
stock.on("message", handle_message)
stock.on("disconnect", handle_disconnect)
stock.on("error", handle_error)

stock.connect()  # WebSocket 連線

行情連接成功
行情連接斷線: 1001, Maximum number of connections reached


Exception: authentication timeout

###### 訂閱商品資料

In [12]:
# 訂閱股票最新成交資訊
stock.subscribe({ 
        "channel": 'trades', 
        "symbol": '1101',
        "intradayOddLot": True,
        })

In [13]:
# 訂閱股票最新最佳五檔委買委賣資訊
stock.subscribe({ 
    'channel': 'books', 
    'symbol': '2330'
})

In [14]:
# 訂閱股票最新指數行情資料
stock.subscribe({ 
    'channel': 'indices', 
    'symbol': 'IR0001'
})

In [15]:
target_id = "Dn90D2VM2McNXqzJx9gpsz5q8A460Qu2MA"  #欲取消訂閱之頻道編號(id)

result = stock.unsubscribe(
        {
          "id": target_id
        }
)

### 斷開 WebSocket 連線

In [2]:
stock.disconnect()

NameError: name 'stock' is not defined

## 行情 WebAPI 查詢

In [1]:
# 建立行情查詢 WebAPI 連線 Object Instance
restStock = sdk.marketdata.rest_client.stock  

NameError: name 'sdk' is not defined

### 日內行情

In [18]:
# 股票或指數列表（依條件查詢）
result = restStock.intraday.tickers(type='EQUITY', exchange="TWSE", market="TSE")
stock_list = ["8467", "9103", "2330"]  # 抽樣查詢之股票 symbols

print(f"資料長度: {len(result['data'])}\n")

for ticker in result["data"]:
    if ticker["symbol"] in stock_list:
        print(ticker)

資料長度: 1384

{'symbol': '2330', 'name': '台積電'}
{'symbol': '8467', 'name': '波力-KY'}
{'symbol': '9103', 'name': '美德醫療-DR'}


In [23]:
# 取得股票資訊 (依股票代碼查詢)
result = restStock.intraday.ticker(symbol='2330')
print(result)
type(result)

{'date': '2024-11-15', 'type': 'EQUITY', 'exchange': 'TWSE', 'market': 'TSE', 'symbol': '2330', 'name': '台積電', 'industry': '24', 'securityType': '01', 'previousClose': 1035, 'referencePrice': 1035, 'limitUpPrice': 1135, 'limitDownPrice': 932, 'canDayTrade': True, 'canBuyDayTrade': True, 'canBelowFlatMarginShortSell': True, 'canBelowFlatSBLShortSell': True, 'isAttention': False, 'isDisposition': False, 'isUnusuallyRecommended': False, 'isSpecificAbnormally': False, 'matchingInterval': 0, 'securityStatus': 'NORMAL', 'boardLot': 1000, 'tradingCurrency': 'TWD'}


dict

In [44]:
# 股票即時報價（依代碼查詢）
result = restStock.intraday.quote(symbol="2330")
stock_name = result.get('name', '未找到股票名稱')  # 使用 get() 方法獲取股票名稱
print(result)
print(f"股票名稱: {stock_name}")
print(f"即時價格: {current_price}")

{'date': '2024-11-15', 'type': 'EQUITY', 'exchange': 'TWSE', 'market': 'TSE', 'symbol': '2330', 'name': '台積電', 'referencePrice': 1035, 'previousClose': 1035, 'openPrice': 1040, 'openTime': 1731632402185024, 'highPrice': 1045, 'highTime': 1731632402218466, 'lowPrice': 1030, 'lowTime': 1731638026106041, 'closePrice': 1035, 'closeTime': 1731648600000000, 'avgPrice': 1037.85, 'change': 0, 'changePercent': 0, 'amplitude': 1.45, 'lastPrice': 1035, 'lastSize': 4823, 'bids': [{'price': 1035, 'size': 1542}, {'price': 1030, 'size': 2752}, {'price': 1025, 'size': 1947}, {'price': 1020, 'size': 1818}, {'price': 1015, 'size': 829}], 'asks': [{'price': 1040, 'size': 321}, {'price': 1045, 'size': 1283}, {'price': 1050, 'size': 1417}, {'price': 1055, 'size': 972}, {'price': 1060, 'size': 612}], 'total': {'tradeValue': 35925255000, 'tradeVolume': 34615, 'tradeVolumeAtBid': 19486, 'tradeVolumeAtAsk': 12842, 'transaction': 7027, 'time': 1731648600000000}, 'lastTrade': {'bid': 1035, 'ask': 1040, 'price': 

In [27]:
# 股票價格Ｋ線（依代碼查詢）
result = restStock.intraday.candles(symbol='2330', timeframe=5)
print(result)

{'date': '2024-11-15', 'type': 'EQUITY', 'exchange': 'TWSE', 'market': 'TSE', 'symbol': '2330', 'timeframe': '5', 'data': [{'date': '2024-11-15T09:00:00.000+08:00', 'open': 1040, 'high': 1045, 'low': 1040, 'close': 1045, 'volume': 3848, 'average': 1041.14}, {'date': '2024-11-15T09:05:00.000+08:00', 'open': 1040, 'high': 1045, 'low': 1040, 'close': 1040, 'volume': 1913, 'average': 1041.21}, {'date': '2024-11-15T09:10:00.000+08:00', 'open': 1040, 'high': 1045, 'low': 1035, 'close': 1040, 'volume': 2486, 'average': 1040.96}, {'date': '2024-11-15T09:15:00.000+08:00', 'open': 1045, 'high': 1045, 'low': 1040, 'close': 1040, 'volume': 286, 'average': 1040.99}, {'date': '2024-11-15T09:20:00.000+08:00', 'open': 1040, 'high': 1045, 'low': 1035, 'close': 1040, 'volume': 797, 'average': 1040.91}, {'date': '2024-11-15T09:25:00.000+08:00', 'open': 1040, 'high': 1040, 'low': 1035, 'close': 1040, 'volume': 105, 'average': 1040.89}, {'date': '2024-11-15T09:30:00.000+08:00', 'open': 1040, 'high': 1040, 

In [28]:
# 股票成交明細（依代碼查詢）
result = restStock.intraday.trades(symbol='2330')
print(result)

{'date': '2024-11-15', 'type': 'EQUITY', 'exchange': 'TWSE', 'market': 'TSE', 'symbol': '2330', 'data': [{'price': 1035, 'size': 110, 'time': 1731652200000000, 'serial': 99999999}, {'bid': 1035, 'ask': 1040, 'price': 1035, 'size': 4823, 'volume': 34615, 'time': 1731648600000000, 'serial': 9860339}, {'bid': 1040, 'ask': 1045, 'price': 1040, 'size': 17, 'volume': 29792, 'time': 1731648297947250, 'serial': 9823333}, {'bid': 1040, 'ask': 1045, 'price': 1045, 'size': 1, 'volume': 29775, 'time': 1731648297833265, 'serial': 9823254}, {'bid': 1040, 'ask': 1045, 'price': 1040, 'size': 1, 'volume': 29774, 'time': 1731648296854469, 'serial': 9822519}, {'bid': 1040, 'ask': 1045, 'price': 1040, 'size': 1, 'volume': 29773, 'time': 1731648296710915, 'serial': 9822423}, {'bid': 1040, 'ask': 1045, 'price': 1040, 'size': 1, 'volume': 29772, 'time': 1731648296554560, 'serial': 9822311}, {'bid': 1040, 'ask': 1045, 'price': 1040, 'size': 1, 'volume': 29771, 'time': 1731648293458771, 'serial': 9819653}, {'b

In [20]:
# 股票分價量表（依代碼查詢）
result = restStock.intraday.volumes(symbol='2330')
print(result)

{'date': '2024-11-13', 'type': 'EQUITY', 'exchange': 'TWSE', 'market': 'TSE', 'symbol': '2330', 'data': [{'price': 1050, 'volume': 4372, 'volumeAtBid': 0, 'volumeAtAsk': 4372}, {'price': 1045, 'volume': 21220, 'volumeAtBid': 8491, 'volumeAtAsk': 8985}, {'price': 1040, 'volume': 9071, 'volumeAtBid': 8827, 'volumeAtAsk': 244}, {'price': 1035, 'volume': 7187, 'volumeAtBid': 486, 'volumeAtAsk': 6701}]}


### 行情快照

In [21]:
# 股票行情快照（依市場別）
result = restStock.snapshot.quotes(market='TSE')
print(result)

{'date': '2024-11-13', 'time': '140000', 'market': 'TSE', 'data': [{'type': 'EQUITY', 'symbol': '0050', 'name': '元大台灣50', 'openPrice': 193.85, 'highPrice': 194, 'lowPrice': 192.5, 'closePrice': 192.75, 'change': -1.25, 'changePercent': -0.64, 'tradeVolume': 9125, 'tradeValue': 1762232150, 'lastPrice': 192.75, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '0051', 'name': '元大中型100', 'openPrice': 80.85, 'highPrice': 80.85, 'lowPrice': 80.15, 'closePrice': 80.45, 'change': -0.4, 'changePercent': -0.49, 'tradeVolume': 95, 'tradeValue': 7644450, 'lastPrice': 80.45, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '0052', 'name': '富邦科技', 'openPrice': 191.5, 'highPrice': 192.2, 'lowPrice': 190.55, 'closePrice': 190.55, 'change': -1.35, 'changePercent': -0.7, 'tradeVolume': 735, 'tradeValue': 140652850, 'lastPrice': 190.55, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '0053', 'name': '元大電子', 'openPrice': 104.25, 'highPrice': 104.5, 'lowPrice': 10

In [22]:
# 股票漲跌幅排行（依市場別）
result = restStock.snapshot.movers(market='TSE', direction='up', change='percent')
print(result)

{'date': '2024-11-13', 'time': '140000', 'market': 'TSE', 'change': 'percent', 'data': [{'type': 'EQUITY', 'symbol': '4968', 'name': '立積', 'openPrice': 170, 'highPrice': 187, 'lowPrice': 168.5, 'closePrice': 187, 'change': 17, 'changePercent': 10, 'tradeVolume': 8318, 'tradeValue': 1529072000, 'lastPrice': 187, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '6282', 'name': '康舒', 'openPrice': 30.35, 'highPrice': 33.7, 'lowPrice': 30.25, 'closePrice': 33.7, 'change': 3.05, 'changePercent': 9.95, 'tradeVolume': 16931, 'tradeValue': 553900000, 'lastPrice': 33.7, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '3018', 'name': '隆銘綠能', 'openPrice': 30.35, 'highPrice': 32.6, 'lowPrice': 30.35, 'closePrice': 32.6, 'change': 2.95, 'changePercent': 9.95, 'tradeVolume': 522, 'tradeValue': 16848950, 'lastPrice': 32.6, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '1618', 'name': '合機', 'openPrice': 42.5, 'highPrice': 44.85, 'lowPrice': 42.4, 'closePric

In [23]:
# 股票成交量值排行（依市場別）
result = restStock.snapshot.actives(market='TSE', trade='volume')
print(result)

{'date': '2024-11-13', 'time': '140000', 'market': 'TSE', 'trade': 'volume', 'data': [{'type': 'EQUITY', 'symbol': '3706', 'name': '神達', 'openPrice': 64.8, 'highPrice': 66.5, 'lowPrice': 63.5, 'closePrice': 63.5, 'change': -0.1, 'changePercent': -0.16, 'tradeVolume': 218506, 'tradeValue': 14150597200, 'lastPrice': 63.5, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '00878', 'name': '國泰永續高股息', 'openPrice': 22.88, 'highPrice': 23, 'lowPrice': 22.78, 'closePrice': 22.92, 'change': 0, 'changePercent': 0, 'tradeVolume': 169780, 'tradeValue': 3890113840, 'lastPrice': 22.92, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '00632R', 'name': '元大台灣50反1', 'openPrice': 3.3, 'highPrice': 3.31, 'lowPrice': 3.28, 'closePrice': 3.3, 'change': 0.01, 'changePercent': 0.3, 'tradeVolume': 117108, 'tradeValue': 386023800, 'lastPrice': 3.3, 'lastUpdated': 1731475800000000}, {'type': 'EQUITY', 'symbol': '00637L', 'name': '元大滬深300正2', 'openPrice': 16.57, 'highPrice': 16.96, 'lo

### 歷史行情

In [27]:
# 取得 1 年內歷史股價（依代碼查詢）
# P.S. 目前分Ｋ無法指定開始日期（from） 與 結束日期（to），一律回傳近五日資料，並且無法選擇 turnover 與 change 的欄位
result = restStock.historical.candles(**{"symbol": "2330", "from": "2023-07-26", "to": "2024-01-30"})
print(result)

{'symbol': '2330', 'type': 'EQUITY', 'exchange': 'TWSE', 'market': 'TSE', 'timeframe': 'D', 'data': [{'date': '2024-01-30', 'open': 642, 'high': 647, 'low': 642, 'close': 642, 'volume': 40398877}, {'date': '2024-01-29', 'open': 646, 'high': 648, 'low': 644, 'close': 648, 'volume': 29828008}, {'date': '2024-01-26', 'open': 644, 'high': 646, 'low': 639, 'close': 644, 'volume': 44103850}, {'date': '2024-01-25', 'open': 635, 'high': 642, 'low': 633, 'close': 642, 'volume': 59214638}, {'date': '2024-01-24', 'open': 628, 'high': 630, 'low': 624, 'close': 627, 'volume': 29905121}, {'date': '2024-01-23', 'open': 629, 'high': 629, 'low': 622, 'close': 628, 'volume': 45761889}, {'date': '2024-01-22', 'open': 633, 'high': 633, 'low': 623, 'close': 626, 'volume': 70829523}, {'date': '2024-01-19', 'open': 625, 'high': 627, 'low': 614, 'close': 626, 'volume': 176166037}, {'date': '2024-01-18', 'open': 586, 'high': 589, 'low': 585, 'close': 588, 'volume': 36746623}, {'date': '2024-01-17', 'open': 583

In [ ]:
# 取得近 52 週股價數據（依代碼查詢）
result = restStock.historical.stats(symbol="2330")
print(result)

# 交易 (單筆)

### 建立委託單

In [24]:
# 定義訂單內容
order = Order(
    buy_sell = BSAction.Buy,
    symbol = "1301",
    price = None,
    quantity = 1000, # 股數; 1000為一張
    market_type = MarketType.Common,
    price_type = PriceType.Reference,
    time_in_force = TimeInForce.ROD,
    order_type = OrderType.Stock,
    user_def = None, # optional field
);

# 列印訂單內容
print(order)

Order {
    buy_sell: Buy,
    symbol: "1301",
    price: None,
    quantity: 1000,
    market_type: Common,
    price_type: Reference,
    time_in_force: ROD,
    order_type: Stock,
    user_def: None,
}


In [25]:
# 下單
order_reponse = sdk.stock.place_order(accounts.data[0], order)
print(order_reponse)

Result {
  is_success: False,
  message: 時間未到，稍待再輸入委託==>[11EFA1CCBB4AD49880000432019B95DE],
  data: None
}


### 取得委託單結果

In [26]:
result = sdk.stock.get_order_results(accounts.data[0])
print(f"筆數: {len(result.data)}")
i = 0
for order_result in result.data:
    print(f"第 {i+1} 筆:")
    print(order_result, end="\n\n")
    i += 1

筆數: 4
第 1 筆:
OrderResult {
    function_type: None,
    date: "2024/11/14",
    seq_no: "00016608372",
    branch_no: "20505",
    account: "37089",
    order_no: None,
    asset_type: 0,
    market: "TAIEX",
    market_type: Common,
    stock_no: "1301",
    buy_sell: Buy,
    price_type: Limit,
    price: 44.5,
    quantity: 2000,
    time_in_force: ROD,
    order_type: Stock,
    is_pre_order: true,
    status: 0,
    after_price_type: None,
    after_price: 44.5,
    unit: 1000,
    after_qty: 2000,
    filled_qty: 0,
    filled_money: 0,
    before_qty: None,
    before_price: None,
    user_def: " ",
    last_time: "15:53:03.105",
    details: None,
    error_message: None,
}

第 2 筆:
OrderResult {
    function_type: None,
    date: "2024/11/14",
    seq_no: "00016608467",
    branch_no: "20505",
    account: "37089",
    order_no: None,
    asset_type: 0,
    market: "TAIEX",
    market_type: Common,
    stock_no: "00940",
    buy_sell: Sell,
    price_type: Limit,
    price: 9.6

#### 使用範例：使用委託書或流水序號取得特定委託單物件

In [27]:
def get_order_by_no(orders, order_or_seq_no, use_order_no=True):
    if orders.data is None:
        print(f"查無委託單資訊")
        return None

    for order in orders.data:
        if use_order_no:  # 以委託書號查詢
            if order.order_no is not None and order.order_no == order_or_seq_no:
                # print(f"提取委託單: {order}")
                return order
        else:  # 以委託單流水序號查詢
            if order.seq_no is not None and order.seq_no == order_or_seq_no:
                # print(f"提取委託單: {order}")
                return order

    # 查無委託單
    field_name = "委託書號" if use_order_no else "委託單流水序號"
    print(f"{field_name} {order_or_seq_no} 查無委託單")
    return None

# 使用範例 1: 以委託書號查詢
orders = sdk.stock.get_order_results(accounts.data[0])  # 查詢所有委託單
order = get_order_by_no(orders, "x0003", use_order_no=True)  # x0016 為目標委託書號
print(f"使用範例 1: 結果委託單 {order}\n")

# # 使用範例 2: 以委託單流水序號查詢
# orders = sdk.stock.get_order_results(accounts.data[0])  # 查詢所有委託單
# order2 = get_order_by_no(orders, "00000237234", use_order_no=False)  # 00000237234 為目標委託單流水序號
# print(f"使用範例 2: 結果委託單 {order2}")

委託書號 x0003 查無委託單
使用範例 1: 結果委託單 None



### 改價

In [ ]:
# 使用範例：使用委託單號取得欲修改之委託單物件
target_order = None
target_order_number = "l0001"  # 欲查找之委託單號

response = sdk.stock.get_order_results(accounts.data[0])

if response.is_success:
    for order in response.data:
        if order.order_no == target_order_number:  # 取第一個狀態為成功的委託單為例
            target_order = order

    if target_order is not None:
        print(target_order)
    else:
        print(f"單號 {target_order_number}，查無委託單")

else:
    print("查無資料")
    print(f"response: {response}")

In [ ]:
# 單筆改價
target_order = order
modify_price_obj = sdk.stock.make_modify_price_obj(target_order, "542",)  # 改價
response = sdk.stock.modify_price(accounts.data[0], modify_price_obj)  # 送出改價單

# 印出回應
# print(f"目標委託單:\n{target_order}\n")
print(f"修改回覆:\n{response}")

### 改量

In [ ]:
#單筆改量
target_order = order
modify_quantity_obj = sdk.stock.make_modify_quantity_obj(target_order, 1000)  # 改量
response = sdk.stock.modify_quantity(accounts.data[0], modify_quantity_obj)  # 送出改量單

# 印出回應
# print(f"目標委託單:\n{target_order}\n")
print(f"修改回覆:\n{response}")

#### 刪單

In [ ]:
# 函數: 刪單 by 單號
def del_order(order_no, account):
    if not isinstance(order_no, str):
        print(f"請輸入單號字串，例如\"x0001\"")
        return None

    # 取得委託單 object
    target_order = None
    orders = sdk.stock.get_order_results(account)

    for order in orders.data:
        if order.order_no == order_no:
            target_order = order

    if target_order is None:
        print(f"查無目標委託單, 目標委託單號 {order_no}. 帳號:\n{account}")
        return None
    else:
        response = sdk.stock.cancel_order(account, target_order)
        print(f"刪單回報:\n{response}\n\n")
        return response

# 刪單
response = del_order("x0005", accounts.data[0])  # 可修改單號及帳號

if response is None:
    print("刪單函數執行錯誤")

else:
    target_order_number = response.data.order_no
    # 取得新的委託單資訊
    target_order = None
    orders = sdk.stock.get_order_results(accounts.data[0])
    for order in orders.data:
        if order.order_no == target_order_number:
            target_order = order
            
    if target_order is None:
        print(f"查無目標委託單。目標委託單號 {target_order_number}")
    else:
        print(f"新委託單資訊(單筆):\n{target_order}\n\nstatus: {target_order.status}")  # status 30 代表刪單成功

### 查詢歷史委託

In [28]:
# 歷史委託
response = sdk.stock.order_history(accounts.data[0], "20240313", "20240313")  # 只供查詢兩日內之歷史資料

if response.is_success:
    
    order_history = response.data
    
    print(f"筆數: {len(order_history)}\n")

    i = 0
    for order_his in order_history:
        i += 1
        print(f"第 {i} 筆:\n{order_his}\n")
        
else:
    print("查尋錯誤")
    print(f"response: {response}")

筆數: 0



### 查詢歷史成交

In [29]:
# 歷史成交
result = sdk.stock.filled_history(accounts.data[0], "20240313", "20240313")  # 只供查詢兩日內之歷史資料
print(result)

Result {
  is_success: True,
  message: None,
  data: []
}


### 資券配額查詢

In [ ]:
# 資券配額查詢
result = sdk.stock.margin_quota(accounts.data[0], "2330")
print(result)

### 現冲券配額查詢

In [ ]:
# 現冲券配額查詢
result = sdk.stock.daytrade_and_stock_info(accounts.data[0], "2330")
print(result)

# 交易 (批次)

### 建立批次委託單

In [ ]:
# 建立欲委託清單
orders = [
  Order(
    buy_sell = BSAction.Buy,
    symbol = "2881",
    price = None,
    quantity = 2000,
    market_type = MarketType.Common,
    price_type = PriceType.LimitDown,
    time_in_force = TimeInForce.ROD,
    order_type = OrderType.Stock,
    user_def = "batch1" # optional field
), Order(
    buy_sell = BSAction.Buy,
    symbol = "1101",
    price = None,
    quantity = 1000,
    market_type = MarketType.Common,
    price_type = PriceType.LimitDown,
    time_in_force = TimeInForce.ROD,
    order_type = OrderType.Stock,
    user_def = "batch2" # optional field
) ]

orders

In [ ]:
# 建立批次委託單
result = sdk.stock.batch_place_order(accounts.data[0], orders)
print(result)

### 取得批次委託送單紀錄

***註：*** 此僅為送單紀錄，無交易狀態更新

In [30]:
# 取得批次委託列表
result = sdk.stock.batch_order_lists(accounts.data[0])
print(result)

Result {
  is_success: True,
  message: None,
  data: []
}


### 取得批次委託送單紀錄明細

***註：*** 此僅為送單紀錄，無交易狀態更新

In [31]:
# 取得批次委託列表
batch_list = sdk.stock.batch_order_lists(accounts.data[0])

# 取得單筆批次委託明細
target_batch_result = batch_list.data[0]  # 單筆批次委託結果
result = sdk.stock.batch_order_detail(accounts.data[0], target_batch_result)

print(result)

IndexError: list index out of range

### 批次修改

In [ ]:
# 查詢委託單
response = sdk.stock.get_order_results(accounts.data[0])

# 取得此前批次下單(同Timestamp)成功之委託單
result_orders = []

if not response.is_success:
    print(f"委託單查詢失敗, 查詢結果:\n{orders}")

else:  # 委託單查詢成功
    for order in response.data:
        if (order.user_def is not None) and (str(timestamp) in order.user_def):
            result_orders.append(order)

# 列印
print(f"取得委託單結果:\n{result_orders}")

#### 批次修改委託價格

In [ ]:
# 建立批次委託修改物件
modify_objects = []

for order in result_orders:
    the_price = order.after_price

    # 設定擬改價格 (自動設定預修改，僅為測試範例)
    if (the_price * 100) % 10 > 0:
        to_be_price = round(the_price + 0.05, 2)
    elif (the_price * 10) % 10 > 0:
        to_be_price = round(the_price + 0.1, 1)
    else:
        to_be_price = round(the_price + 1)

    # 建立改價 obj
    the_modify_price_obj = sdk.stock.make_modify_price_obj(order, str(to_be_price))
    print(f"委託單編號 {order.order_no}, 現價 {the_price}, 擬改價 {to_be_price}")

    # 放入批次列表
    modify_objects.append(the_modify_price_obj)

# 開始批次改價
if len(modify_objects) > 0:
    modified_results = sdk.stock.batch_modify_price(accounts.data[0], modify_objects)
    
    # 處理批次修改委託價格回傳結果
    if modified_results.is_success:
        print("修改成功")
        print(modified_results.data)
    else:
        print("修改失敗", modified_results.message)

else:
    print("改價列表為空白")

#### 批次修改委託數量

In [ ]:
# 查詢委託單
response = sdk.stock.get_order_results(accounts.data[0])

# 取得此前批次下單(同Timestamp)成功之委託單
result_orders = []

if not response.is_success:
    print(f"委託單查詢失敗, 查詢結果:\n{orders}")

else:  # 委託單查詢成功
    for order in response.data:
        if (order.user_def is not None) and (str(timestamp) in order.user_def):
            result_orders.append(order)

# 列印
print(f"取得委託單結果:\n{result_orders}")

In [ ]:
# 建立批次委託修改物件
modify_objects = []

for order in result_orders:
    the_qty = order.after_qty

    if the_qty > 1000:
        to_be_qty = the_qty - 1000
        
        # 建立改價 obj
        the_modify_price_obj = sdk.stock.make_modify_quantity_obj(order, int(to_be_qty))
        print(f"委託單編號 {order.order_no}, 現量 {the_qty}, 擬改量 {to_be_qty}")
    
        # 放入批次列表
        modify_objects.append(the_modify_price_obj)

    else:
        print(f"委託單編號 {order.order_no}, 現量 {the_price}, 現量不足，略過")

# 開始批次改量
if len(modify_objects) > 0:
    modified_results = sdk.stock.batch_modify_quantity(accounts.data[0], modify_objects)
    
    # 處理批次修改委託量回傳結果
    if modified_results.is_success:
        print("修改成功")
        print(modified_results.data)
    else:
        print("修改失敗", modified_results.message)

else:
    print("改價列表為空白，出問題了!")

#### 刪除批次委託單

In [ ]:
# 查詢委託單
response = sdk.stock.get_order_results(accounts.data[0])

# 取得此前批次下單(同Timestamp)成功之委託單
result_orders = []

if not response.is_success:
    print(f"委託單查詢失敗, 查詢結果:\n{orders}")

else:  # 委託單查詢成功
    for order in response.data:
        if (order.user_def is not None) and (str(timestamp) in order.user_def):
            result_orders.append(order)

# 列印
print(f"取得委託單結果:\n{result_orders}")

In [ ]:
for order in result_orders:
    print(f"擬刪除之委託單編號 {order.order_no}\n")

# 開始批次刪除
if len(result_orders) > 0:
    cancel_result = sdk.stock.batch_cancel_order(accounts.data[0], result_orders)
    
    # 處理批次刪除回傳結果
    if cancel_result.is_success:
        print("刪除成功")
        print(cancel_result.data)
    else:
        print("刪除失敗", cancel_result.message)

# 帳務

In [32]:
# 顯示帳號資訊
accounts.data[0]

Account {
    name: "李定洲",
    branch_no: "20505",
    account: "37089",
    account_type: "stock",
}

### 庫存查詢

In [33]:
# 庫存查詢
result = sdk.accounting.inventories(accounts.data[0])

if result.is_success:
    print(f"資料筆數: {len(result.data)}\n")
    i = 0
    for inv in result.data:
        i += 1
        print(f"第 {i} 筆\n")
        print(f"{inv}\n")
        
else:
    print("查詢失敗")
    print(result)

資料筆數: 5

第 1 筆

Inventory {
    date: "2024/11/14",
    account: "37089",
    branch_no: "20505",
    stock_no: "00919",
    order_type: Stock,
    lastday_qty: 35000,
    buy_qty: 0,
    buy_filled_qty: 0,
    buy_value: 0,
    today_qty: 35000,
    tradable_qty: 35000,
    sell_qty: 0,
    sell_filled_qty: 0,
    sell_value: 0,
    odd: InventoryOdd {
        lastday_qty: 0,
        buy_qty: 0,
        buy_filled_qty: 0,
        buy_value: 0,
        today_qty: 0,
        tradable_qty: 0,
        sell_qty: 0,
        sell_filled_qty: 0,
        sell_value: 0,
    },
}

第 2 筆

Inventory {
    date: "2024/11/14",
    account: "37089",
    branch_no: "20505",
    stock_no: "00940",
    order_type: Stock,
    lastday_qty: 100000,
    buy_qty: 0,
    buy_filled_qty: 0,
    buy_value: 0,
    today_qty: 100000,
    tradable_qty: 100000,
    sell_qty: 0,
    sell_filled_qty: 0,
    sell_value: 0,
    odd: InventoryOdd {
        lastday_qty: 0,
        buy_qty: 0,
        buy_filled_qty: 0,
 

### 未實現損益

In [34]:
# 未實現損益
result = sdk.accounting.unrealized_gains_and_loses(accounts.data[0])
print(result)

Result {
  is_success: True,
  message: None,
  data: [UnrealizedData {
    date: "2024/11/13",
    branch_no: "20505",
    account: "37089",
    stock_no: "00919",
    buy_sell: Buy,
    order_type: Stock,
    cost_price: 23.419,
    tradable_qty: 35000,
    today_qty: 35000,
    unrealized_profit: 0,
    unrealized_loss: 11030,
}, UnrealizedData {
    date: "2024/11/13",
    branch_no: "20505",
    account: "37089",
    stock_no: "00940",
    buy_sell: Buy,
    order_type: Stock,
    cost_price: 9.2524,
    tradable_qty: 100000,
    today_qty: 100000,
    unrealized_profit: 21457,
    unrealized_loss: 0,
}, UnrealizedData {
    date: "2024/11/13",
    branch_no: "20505",
    account: "37089",
    stock_no: "1101",
    buy_sell: Buy,
    order_type: Stock,
    cost_price: 33.156,
    tradable_qty: 40000,
    today_qty: 40000,
    unrealized_profit: 0,
    unrealized_loss: 33984,
}, UnrealizedData {
    date: "2024/11/13",
    branch_no: "20505",
    account: "37089",
    stock_no: "13

### 已實現損益

In [1]:
# 已實現損益
result = sdk.accounting.realized_gains_and_loses(accounts.data[0])
print(result)

NameError: name 'sdk' is not defined

### 維持率查詢

In [ ]:
# 維持率
result = sdk.accounting.maintenance(accounts.data[0])
print(result)

### 交割款查詢

In [ ]:
# 交割款
result = sdk.accounting.query_settlement(accounts.data[0],"3d")
print(result)

### 銀行餘額查詢

In [ ]:
# 銀行餘額
result = sdk.accounting.bank_remain(accounts.data[0])
print(result)

# 交易/帳務 主動回報

In [35]:
import time
import os 

# A callback to receive quote data
def on_order(err, content):
    print("==下單主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_order(on_order) 

# A callback to receive quote data
def on_order_changed(err, content):
    print("==改單主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_order_changed(on_order_changed) 

def on_filled(err, content):
    print("==成交主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_filled(on_filled)

# A callback to receive quote data
def on_event(err, content):
    print("==事件主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_event(on_event) 